### Imports

In [49]:
import os
import joblib
import numpy as np
import pandas as pd
import importlib
import re
import yaml

import sklearn.ensemble 
import sklearn.model_selection
import sklearn.inspection

import findatree.io as io
import findatree.descriptions as descriptions

# Dictionaries: species_name to ba and vice versa
species_id_to_name = descriptions.species_id_to_name()
species_name_to_id = descriptions.species_name_to_id()

### Definitions

In [50]:
# Directory: Processed tnr%.hdf5s
dir_hdf5 = r"C:\Data\lwf\processed\2020\hdf5"

# Directory: Plots
dir_plots = r"C:\Data\lwf\analysis\220830_random-forrest\plots"

# Directory: sklearn
dir_sklearn = r"C:\Data\lwf\analysis\220830_random-forrest\sklearn\v01"

# Save names:
save_name_params = 'params.yaml'
save_name_gridcv = 'grid.joblib'
save_name_dataset = 'dataset.joblib'
save_name_permutation_test_score = 'permutation_test_score.joblib'
save_name_permutation_feature_importance = 'permutation_feature_importance.joblib'

### Load

#### Load: features

In [51]:
importlib.reload(io)
df, params_df = io.allhdf5s_crowns_features_to_dataframe(dir_hdf5, crowns_type='crowns_human')

#### Assign families: Conifers and Broadleaf

In [52]:
importlib.reload(descriptions)

# Define families by patterns
family_patterns = [
    'kiefer|fichte|tanne|douglasie|lärche', 
    'buche|eiche|ahorn|erle|birke|esche',
]

family_names = [
    'conifers',
    'broadleaf',
]

families = descriptions.species_groupby_families(family_patterns, family_names)
family_ids = descriptions.species_id_to_family_id(df.ba.values, families)

df = df.assign(
    family = family_ids,
    )

#### Clean-up: features
* `inf` and `nan` removal
* Filtering based on `['kkl', 'equivalent_diameter_area', 'min_bright_ndre', 'eccentricity']`

In [53]:
# Replace inf values with nans
df = df.where(df != np.inf, np.nan)

# Get rows containing missing values
rows_isnull = df.isnull().any(axis=1)
print(f"{np.sum(rows_isnull)} of {len(df)} crowns: At least one NaN in features")

# Remove these rows
df = df.loc[~rows_isnull, :]
print(f"After NaN removal: Shape of df = {df.shape}")

# Filtering
query_str = 'kkl in [1, 2, 3]'
query_str += ' and equivalent_diameter_area > 1'
query_str += ' and min_bright_ndre > 0'
query_str += ' and eccentricity > 0'
query_str += ' and eccentricity < 1'

df = df.query(query_str)
print(f"After filtering: Shape of df = {df.shape}")

48 of 3895 crowns: At least one NaN in features
After NaN removal: Shape of df = (3847, 281)
After filtering: Shape of df = (3716, 281)


### Infos

#### Search: Pattern in column names

In [54]:
pattern = '^x_|^y'

cols = list(df.columns)
for col in cols:
    if bool(re.search(pattern, col, re.IGNORECASE)):
        print(col)

x_max_chm
x_max_light
y_max_chm
y_max_light
x_mean
y_mean
x_min_bbox
x_max_bbox
y_min_bbox
y_max_bbox
x_min_bbox_bright
x_max_bbox_bright
y_min_bbox_bright
y_max_bbox_bright


#### Info: Dataset

In [55]:
importlib.reload(descriptions)

descriptions.print_summary(
    df.tnr.values,
    df.ba.values,
    df.family.values,
    families,
)

Total number of crowns        : 3716
Mean number of crowns per tnr : 29.7
__________________________________________________

species_id| species_name                  | count
--------------------------------------------------
       134| Gemeine Kiefer                : 1517
       118| Gemeine Fichte                : 888
        20| Rotbuche                      : 425
        48| Traubeneiche                  : 199
       100| Weißtanne                     : 186
       116| Europäische Lärche            : 108
        51| Stieleiche                    : 81
         7| Schwarzerle                   : 38
        22| Gemeine Esche                 : 37
        10| Gemeine Birke                 : 36
       136| Douglasie                     : 33
        13| Hainbuche                     : 32
         5| Bergahorn                     : 23
       129| Schwarzkiefer                 : 16
        36| Kirsche                       : 14
         1| Feldahorn                     : 14
        53| Ro

### Classification

#### Overall parameters

In [56]:
params = {
    'query': query_str,
    'classes':families,
    'test_size': 0.25,
    'cv_splits': 5,
    'shuffle': True,
    'scoring': 'accuracy',
    'n_permutations': 20,
    'n_repeats': 10,
    'max_samples': 0.5,
}

# Save parameters
io.list_of_dicts_to_yaml(os.path.join(dir_sklearn, save_name_params), [params])

#### Define: Features and labels
* Exclude terrestrial features
* Exclude coordinates
* Exclude non-family members

Create extended labels `y_extend` with information about `['family', 'ba', 'sst', 'nbv']`, to check later if classficiation is dependent on these.

In [57]:
# Terrestrial and identifaction feature names to be excluded
terr_names = [
    'id', 'tnr', 'family',
    'enr', 'bnr', 'ba', 'bhd_2020', 
    'alter_2020', 'bk', 'kkl', 'nbv',
    'sst', 'gilb', 'kommentar', 'sicherheit',
]

# Coordinate features pattern to be excluded
coordinate_pattern = '^x_|^y_'

# Get all columns
x_names = list(df.columns)

# Exclude terrestrial feature names
x_names = [col for col in x_names if col not in terr_names]

# Exclude coordinates
x_names = [col for col in x_names if not bool(re.search(coordinate_pattern, col))]

# Define features -> x
x = df.loc[df.family >= 0, x_names].values

# Define labels -> y
y_names = ['family']
y = df.loc[df.family >= 0, y_names[0]].values

# Define extended labels -> y_extend
y_extend_names = ['family', 'tnr', 'id', 'ba', 'sst', 'nbv']

y_extend = df.loc[df.family >= 0, y_extend_names].values

print(f"x.shape: {x.shape}")
print(f"y.shape: {y.shape}")
print(f"y_extend.shape: {y_extend.shape}")
print(f"Ratio in y: [#class=0]/[#class=1]: {np.sum(y == 0) / np.sum(y == 1):.1f}")

x.shape: (3661, 252)
y.shape: (3661,)
y_extend.shape: (3661, 6)
Ratio in y: [#class=0]/[#class=1]: 3.1


#### Train-test split

In [58]:
x_train, x_test, y_train, y_test, y_extend_train, y_extend_test, = sklearn.model_selection.train_test_split(
    x,
    y,
    y_extend,
    test_size=params['test_size'],
    shuffle=True,
    stratify=y,
)

dataset = {
    'x_names': x_names,
    'y_names': y_names,
    'y_extend_names': y_extend_names,
    'x_train': x_train,
    'x_test': x_test,
    'y_train': y_train,
    'y_test': y_test,
    'y_extend_train': y_extend_train,
    'y_extend_test': y_extend_test,
}

# Save dataset
joblib.dump(dataset, os.path.join(dir_sklearn, save_name_dataset)) 

['C:\\Data\\lwf\\analysis\\220830_random-forrest\\sklearn\\v01\\dataset.joblib']

#### Grid Search

In [59]:
params_grd = {
    'criterion': ['gini', 'entropy'],
    'n_estimators': [50, 100, 200],
    'min_samples_leaf': [1, 3, 5],
    'max_samples': [0.75, 0.5]
}
grd = sklearn.model_selection.GridSearchCV(
    sklearn.ensemble.RandomForestClassifier(n_jobs=-1),
    param_grid = params_grd,
    scoring  = params['scoring'],
    cv = sklearn.model_selection.KFold(n_splits=params['cv_splits'], shuffle=params['shuffle'])
)
grd.fit(x_train, y_train)

# Select best_estimator and retrain on complete training set
rfc = grd.best_estimator_
rfc.fit(x_train, y_train)

# Save gridcv
joblib.dump(grd, os.path.join(dir_sklearn, save_name_gridcv))

# Best estimator
print(f"grd.best_params_: {grd.best_params_}")
print(f"grd.best_estimator_.score(x_test, y_test): {grd.best_estimator_.score(x_test, y_test):.3f}")

grd.best_params_: {'criterion': 'entropy', 'max_samples': 0.75, 'min_samples_leaf': 1, 'n_estimators': 200}
grd.best_estimator_.score(x_test, y_test): 0.925


#### Best Estimator: Permutation Test Score

In [60]:
score, permutation_score, pvalue = sklearn.model_selection.permutation_test_score(
    grd.best_estimator_,
    x_train,
    y_train,
    cv=sklearn.model_selection.KFold(n_splits=params['cv_splits'], shuffle=params['shuffle']),
    n_permutations=params['n_permutations'],
    n_jobs=-1,
)

# Add result to params
permutation_test_score = {
    'permutation_scores':  permutation_score,
    'test_score': score,
    'pvalue': pvalue,
}

# Save permutation_test_score
joblib.dump(permutation_test_score, os.path.join(dir_sklearn, save_name_permutation_test_score))

['C:\\Data\\lwf\\analysis\\220830_random-forrest\\sklearn\\v01\\permutation_test_score.joblib']

#### Best Estimator: Permutation Feature Importance

In [61]:
perm_imp_train = sklearn.inspection.permutation_importance(
    grd.best_estimator_,
    x_test,
    y_test,
    n_repeats=params['n_repeats'],
    max_samples=params['max_samples'],
)

# Sort according to importance
sort_idx = np.flip(np.argsort(np.abs(perm_imp_train['importances_mean'])))

# Add result to params
permutation_feature_importance = {
    'on': 'test',
    'importances_mean': perm_imp_train['importances_mean'],
    'importances_std': perm_imp_train['importances_std'],
    'importances': perm_imp_train['importances'],
}

# Save permutation feature importances
joblib.dump(permutation_feature_importance, os.path.join(dir_sklearn, save_name_permutation_feature_importance))

['C:\\Data\\lwf\\analysis\\220830_random-forrest\\sklearn\\v01\\permutation_feature_importance.joblib']

### Load: Previous results

In [62]:
load_name = save_name_gridcv
grd = joblib.load(os.path.join(dir_sklearn, load_name))

load_name = save_name_params
with open(os.path.join(dir_sklearn, load_name), "r") as f:
    params = yaml.safe_load(f)